# Майнор "Прикладные задачи анализа данных"
## Домашнее задание 1 [10 баллов] до 23:59 17.02.2018

В этом домашнем задании вам потребуется написать генератор описания прогноза погоды на следующую неделю в каком-нибудь городе. Домашнее задание состоит из трех частей:
1. Скачивание данных о состоянии погоды в городе с gismeteo.ru
2. Генерация описания прогноза
3. Творческая часть

Все три части можно считать независимыми – вы можете сделать одну или две из них, однако мы настоятельно советуем выполнить все три. Все инструкции по выполнению домашнего задания – ниже. 



### 1. Сбор данных [3 балла]

Выберите произвольным образом город в России и найдите прогноз погоды в нем на ближайшие 10 дней на сайте gismeteo.ru.

Пример: прогноз на 10 ближайших дней в Москве – https://www.gismeteo.ru/weather-moscow-4368/10-days/

Используя известные вам библиотеки для работы с протоколом http и html кодом, извлеките прогноз на ближайшие 10 дней, начиная со дня, когда вы начали делать домашнее задание.  

Резльтатом сбора данных должна быть таблица со следующими строками:
* минимальная температура
* максимальна температура
* скорость ветра
* уровень осадков 

В столбцах таблицы должны быть даты и дни недели.  Пример итоговой таблицы вы найдете в следующей части задания. 

Сделаем import всех необходимых библиотек.

In [1]:
from scrapy.crawler import CrawlerProcess
from twisted.internet.error import ReactorNotRestartable
import scrapy
import random
import pandas as pd
from json import dump
import datetime
import re

Выберем случайный российский город. Данные взяты из [Википедии](https://en.wikipedia.org/wiki/List_of_cities_and_towns_in_Russia).

In [2]:
CITIES_URL = 'https://en.wikipedia.org/wiki/List_of_cities_and_towns_in_Russia'
CITY_SELECTOR = '//tr/td[2]'

In [3]:
cities = []
class RussianCities(scrapy.Spider):
    name = 'russian_cities'
    start_urls = [CITIES_URL,]
    def parse(self, response):
        for city in response.xpath(CITY_SELECTOR):
            cities.append(city.xpath('text()').extract_first())

In [4]:
process = CrawlerProcess()
process.crawl(RussianCities)
process.start()

2018-02-13 02:12:03 [scrapy.utils.log] INFO: Scrapy 1.5.0 started (bot: scrapybot)
2018-02-13 02:12:03 [scrapy.utils.log] INFO: Versions: lxml 4.1.1.0, libxml2 2.9.7, cssselect 1.0.3, parsel 1.4.0, w3lib 1.19.0, Twisted 17.9.0, Python 2.7.14 (default, Jan  5 2018, 10:41:29) - [GCC 7.2.1 20171224], pyOpenSSL 17.5.0 (OpenSSL 1.1.0g  2 Nov 2017), cryptography 2.1.4, Platform Linux-4.14.15-gnu-1-x86_64-with-glibc2.2.5
2018-02-13 02:12:03 [scrapy.crawler] INFO: Overridden settings: {}
2018-02-13 02:12:03 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.logstats.LogStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.corestats.CoreStats']
2018-02-13 02:12:03 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 'scrapy.downloadermiddlewares.downloadtimeout.DownloadTimeoutMiddleware',
 'scrapy.downloadermiddlewares.defaultheaders.DefaultHeadersMiddlew

In [5]:
len(cities)

1112

In [6]:
random.seed(4118956)
print(random.choice(cities))

Сергиев Посад


Выбрали Сергиев Посад. 

In [1]:
CITY_URL = 'https://www.gismeteo.ru/weather-sergiyev-posad-11958/10-days/'

SELECTORS = {
    'weekday': '//div[@class="forecast_frame"]//div[@class="w_date__day"]/text()',
    'day': '//span[contains(@class, "w_date__date")]/text()',
    'min_temp': '//div[@class="maxt"]/text()',
    'max_temp': '//div[@class="mint"]/text()',
    'wind_speed': '//div[contains(@class,"w_wind__warning")]/text()',
    'precipitation': '//div[contains(@class,"w_precipitation__value")]/text()',
}

def process_digit(d):
    '''
    Removes all junk like \n and whitespaces.
    Returns: pure float number.
    '''
    return re.findall('[\d,]+', d)[0]

Данные о погоде будут собираться по состоянию на 12.02.2018

In [2]:
json_data = []
class GismeteoForecast(scrapy.Spider):
    name = 'gismeteo_forecast'
    start_urls = [CITY_URL,]
    def parse(self, response):
        weekdays = response.xpath(SELECTORS['weekday']).extract()
        days = response.xpath(SELECTORS['day']).extract()[:10]
        min_temp = response.xpath(SELECTORS['min_temp']).extract()[:10]
        max_temp = response.xpath(SELECTORS['max_temp']).extract()[:10]
        wind_speed =  response.xpath(SELECTORS['wind_speed']).extract()[:10]
        precipitation = response.xpath(SELECTORS['precipitation']).extract()
        for i in range(10):
            json_data.append({
                'weekday': weekdays[i],
                'day': process_digit(days[i]),
                'minTemp': min_temp[i],
                'maxTemp': max_temp[i],
                'windSpeed': process_digit(wind_speed[i]),
                'precipitation': process_digit(precipitation[i]),
            })

In [3]:
try:
    process = CrawlerProcess()
    process.crawl(GismeteoForecast)
    process.start()
except ReactorNotRestartable:
    print('\n\n\n------------\nRestart kernel and rerun this cell.\n------------')

2018-02-13 02:12:41 [scrapy.utils.log] INFO: Scrapy 1.5.0 started (bot: scrapybot)
2018-02-13 02:12:41 [scrapy.utils.log] INFO: Versions: lxml 4.1.1.0, libxml2 2.9.7, cssselect 1.0.3, parsel 1.4.0, w3lib 1.19.0, Twisted 17.9.0, Python 2.7.14 (default, Jan  5 2018, 10:41:29) - [GCC 7.2.1 20171224], pyOpenSSL 17.5.0 (OpenSSL 1.1.0g  2 Nov 2017), cryptography 2.1.4, Platform Linux-4.14.15-gnu-1-x86_64-with-glibc2.2.5
2018-02-13 02:12:41 [scrapy.crawler] INFO: Overridden settings: {}
2018-02-13 02:12:41 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.logstats.LogStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.corestats.CoreStats']
2018-02-13 02:12:41 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 'scrapy.downloadermiddlewares.downloadtimeout.DownloadTimeoutMiddleware',
 'scrapy.downloadermiddlewares.defaultheaders.DefaultHeadersMiddlew

Сохраним результаты в json.

In [4]:
with open('result.json', 'w') as fp:
    dump(json_data, fp)

Считаем в dataframe записанный json-файл.

In [5]:
df = pd.read_json('./result.json')

In [6]:
df

,day,maxTemp,minTemp,precipitation,weekday,windSpeed
0,13,−13,−7,0,Вт,7
1,14,−14,−8,0,Ср,8
2,15,−13,−8,"0,2",Чт,7
3,16,−8,−4,3,Пт,10
4,17,−5,−2,"3,8",Сб,10
5,18,−4,−1,"0,8",Вс,8
6,19,−3,−1,"0,8",Пн,9
7,20,−2,+3,"8,8",Вт,15
8,21,−4,+3,"0,2",Ср,15
9,22,−9,−4,0,Чт,5


Зададим таблице правильный индекс и транспонируем.

In [7]:
days = df['day']
weekdays = df['weekday']
new_index = []

for day, w_day in zip(days, weekdays):
    new_index.append('{d}.0{m} ({w})'.format(d=day,
                                            w=w_day.encode('utf-8'),
                                            m=datetime.date.today().month))

df.drop(['day', 'weekday'], axis=1, inplace=True)
df.index = new_index
df = df.T

In [8]:
df

,13.02 (Вт),14.02 (Ср),15.02 (Чт),16.02 (Пт),17.02 (Сб),18.02 (Вс),19.02 (Пн),20.02 (Вт),21.02 (Ср),22.02 (Чт)
maxTemp,−13,−14,−13,−8,−5,−4,−3,−2,−4,−9
minTemp,−7,−8,−8,−4,−2,−1,−1,+3,+3,−4
precipitation,0,0,"0,2",3,"3,8","0,8","0,8","8,8","0,2",0
windSpeed,7,8,7,10,10,8,9,15,15,5


In [9]:
df.to_csv('gismeteo_forecast.csv', sep=';', encoding='utf-8')

### 2. Генератор описания прогноза погоды [4 балла]

Если у вас не получилось извлечь прогноз погоды в предыдущей части задания, воспользуйтесь таблицей ниже.
В ней приведен прогноз четырех показателей на ближайшие 10 дней в Москве – минимальная и максимальная температура, скорость ветра и уровень осадков. 

|                | 02.02 (пт) | 03.02 (сб) | 04.02 (вс)| 05.02 (пн) | 06.02 (вт) | 07.02 (пн) | 08.02 (ср) | 09.02 (ср) | 10.02 (сб) | 11.02 (вс)
|----------------|-------|-------|-------|-------|-------|-------|-------|
| минимальная температура    | -9    | -1    | -8    | -13    | -12    | -15    | -21    | -14 |-8 |-8
| максимальная температура    | -1    | +1    | -2    | -9   | -11    | -12    | -16    |-5    |-6    |-5|
| скорость ветра | 10    | 13    | 15    | 15   |11    | 6    | 7 | 9 | 8 |12
| уровень осадков         | 1.35  | 8.6  | 15.5  | 6.6   | 2.7   | 2.1   | 0   | 3.2   |0.8  | 0.4

Прогноз погоды должен состоять из следующих (или подобным им) предложений, генерируемых по шаблонам (ниже три шаблона):
* В день1 похолодает / потеплеет на X градус (-а, -ов) по сравнению с день2
    * *В четверг в НазваниеГорода потеплеет на 7 градусов по сравнению со средой*
* Скорость ветра изменится на X единиц в день1 по сравнению с день2.
    * *Скорость ветра изменится на 3 единицы в понедельник по сравнению с пятницей*
* Уровень осадков повысится / понизится на X единиц за Y дней. 
    * *Уровень осадков понится на 3.85 единиц за 7 дней*

Вместо НазваниеГорода подставьте название выбранного вами города, используя фунцкии для согласования существительных с предлогами. 

Некоторые вспомогательные функции, которые вам понадобятся: согласование существительного с числительным и приведение существительного к нужному падежу: 

In [4]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
word = 'яблоко'
parsed_word = morph.parse(word)[0]
print(1, parsed_word.make_agree_with_number(1).word) # согласование слова с числительным 1
print(2, parsed_word.make_agree_with_number(2).word) # согласование слова с числительным 2
print(3, parsed_word.make_agree_with_number(5).word) # согласование слова с числительным 5

1 яблоко
2 яблока
3 яблок


In [6]:
print(parsed_word.inflect({'gent'}).word) # слово в родительном падеже

яблока


Ниже приведен пример решения аналогичной задачи генерации текстов: генератор отчета о том, сколько и каких фруктов съел Вася.

In [17]:
def eating(n, fruit):
    parsed_word = morph.parse(fruit)[0]
    s = ' '.join(['Вася съел', str(n) ,parsed_word.make_agree_with_number(n).word, '.'])
    return s

In [18]:
eating(5, 'яблоко')

'Вася съел 5 яблок .'

In [19]:
eating(4, 'груша')

'Вася съел 4 груши .'

Пример более сложного генератора: фрукты могут есть не только мальчики, но и девочки, то есть, нужно не только согласовать числительное с существительным, но и поставить глагол в нужную форму.

In [27]:
def eating(name, n, fruit):
    name_tag = morph.parse(name)[0].tag
    if 'masc' in name_tag:
        gender = 'masc'
    if 'femn' in name_tag:
        gender = 'femn'
    verb = morph.parse("съесть")[0].inflect({'perf', gender,'sing','past','indc'}).word
    parsed_word = morph.parse(fruit)[0]
    s = ' '.join([name, verb, str(n) ,parsed_word.make_agree_with_number(n).word, '.'])
    return s

In [28]:
eating('Маша', 3, 'вишня')

'Маша съела 3 вишни .'

### 3. Ответьте на вопросы [3 балла]
* В каких других задачах (помимо описания прогноза погоды) может понадобиться генерировать текст по шаблонам? В каких задачах может понадобиться генерировать текст об изменении числовых показателей по шаблонам?
* Шаблоны, которые вы использовали в этом задании, имеют фиксированную структуру. Фактически, ваша задача заключалась в том, чтобы подставить в шаблон число и согласовать единицы измерения с этим числом или подставить в шаблон название города и согласовать его с предлогом. Как можно разнообразить эти шаблоны? Как знание синтаксической структуры предложения может помочь в этой задаче? 

## Сдача домашнего задания

Дедлайн сдачи домашнего задания:  23:59 17.02.2018.

Результаты домашнего задания должны быть оформлены в виде отчета в jupyter notebook.
Нормальный отчёт должен включать в себя:
* Краткую постановку задачи и формулировку задания
* Описание минимума необходимой теории и/или описание используемых инструментов 
* Подробный пошаговый рассказ о проделанной работе
* **Аккуратно** оформленные результаты
* Подробные и внятные ответы на все заданные вопросы 
* Внятные выводы – не стоит относится к домашнему заданию как к последовательности сугубо технических шагов, а стоит относится скорее как к небольшому практическому исследованию, у которого есть своя цель и свое назначение.

Задание выполняется в группе до трех человек. Не забудьте перечислить фамилии всех, кто работал над домашнем задании, в jupyter notebook.  

В случае использования какого-либо строннего источника информации обязательно дайте на него ссылку (поскольку другие тоже могут на него наткнуться). Плагиат наказывается нулём баллов за задание и предвзятым отношением в будущем.


При возникновении проблем с выполнением задания обращайтесь с вопросами к преподавателю по семинарским занятиям в вашей группе или у учебным ассистентам.

Учебный ассистент по ДЗ 1: Анна Лапидус (email: anyalapidus@list.ru, telegram: @alpids).


Небрежное оформление отчета существенно отразится на итоговой оценке. Весь код из отчёта должен быть воспроизводимым, если для этого нужны какие-то дополнительные действия, установленные модули и т.п. — всё это должно быть прописано в отчете в явном виде.

Сдача отчетов осуществляется через систему AnyTask.

### Как сдать домашнее задание в AnyTask
* Зарегистрируйтесь в системе AnyTask по ссылке http://anytask.org/accounts/register . Регистрация обязательна для всех!
* Подтвердите регистрацию по e-mail.
* Зайдите в свой профиль, нажмите “Активация инвайтов на курсы” и введите инвайт:

ИАД-1 : VNYQlGd

ИАД-2 : wDEa8ge

ИАД-3 : SviK5dd

ИАД-4 : dzdD8YG


 У вас появится курс “ВШЭ > Прикладные задачи анализа данных” в разделе “Посещает курсы”.
* Перейдите по ссылке “ВШЭ > Прикладные задачи анализа данных (2018)” и нажмите кнопку “Сдать”. 
* У вас откроется условие задачи и будут доступны различные поля, в частности, НИЖЕ условия задачи будет поле ввода, в которое вы сможете вписать какой-то комментарий, и сможете прикрепить файл. Сделайте это.
* Домашнее задание лучше всего сдавать в форматах IPYNB.
* Оценку вы получите также в системе AnyTask. За своей успеваемостью можете следить в разделе “Ведомость”, а также можете прокомментировать что-то в каждом вашем домашнем задании, зайдя на ее страничку (ячейки в табличке на страничке “Ведомость” кликабельны и ведут на ваш submission домашки).

(**ВАЖНО**) Если домашнее задание вы делали в группе, то в AnyTask домашнее сдает *один* участник группы, но заргестрироваться в AnyTask обязательно всем – так мы сможем проставить вам оценки в ведомость в AnyTask.

Ссылка на курс в AnyTask: http://anytask.org/course/281